In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import jax
jax.config.update('jax_disable_jit', False) # Turn off JIT because of an issue in shortwave_radiation.py:169
jax.config.update("jax_debug_infs", True) # doesn't add any time since the saved time is otherwise spent getting the nodal quantities
jax.config.update("jax_debug_nans", False) # some physics fields might be nan

In [3]:
from jcm.model import Model, get_coords
from jcm.physics.icon.icon_physics import IconPhysics
from jcm.boundaries import initialize_boundaries

realistic_boundaries = lambda coords=get_coords(): initialize_boundaries(
    '../jcm/data/bc/t30/clim/boundaries_daily.nc',
    coords.horizontal,
)

Platform 'METAL' is experimental and not all JAX functionality may be correctly supported!
W0000 00:00:1754700447.268905  427418 mps_client.cc:510] WARNING: JAX Apple GPU support is experimental and not all JAX functionality is correctly supported!
I0000 00:00:1754700447.422321  427418 service.cc:145] XLA service 0x7fa21ca844a0 initialized for platform METAL (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1754700447.422676  427418 service.cc:153]   StreamExecutor device (0): Metal, <undefined>
I0000 00:00:1754700447.438411  427418 mps_client.cc:406] Using Simple allocator.
I0000 00:00:1754700447.438783  427418 mps_client.cc:384] XLA backend will use up to 5726208000 bytes on device 0 for SimpleAllocator.


Metal device set to: Apple M1

systemMemory: 8.00 GB
maxCacheSize: 2.67 GB



In [6]:
model = Model(
    save_interval=3,
    total_time=3/48,
    layers=40,
    physics=IconPhysics(radiation="rrtmgp") # "icon" or "rrtmgp"
    # boundaries=realistic_boundaries(),
)
state = model.get_initial_state()
final_state, predictions = model.unroll(state)